In [31]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import classification_report
import re
import pandas as pd 
import random


In [22]:
songs=pd.read_csv('labeled_lyrics_cleaned.csv')
songs.head()

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [23]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    text = re.sub('[\r\n]', '', text)
    return text

In [5]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

tweets = np.concatenate([positive_tweets, negative_tweets])
labels = np.concatenate([np.ones(len(positive_tweets)), np.zeros(len(negative_tweets))])

tokenizer = TweetTokenizer(preserve_case=False)
tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in tweets]



In [6]:
vocab_size = 10000
max_length = 30
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(tokenized_tweets)
sequences = tokenizer.texts_to_sequences(tokenized_tweets)

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)



In [7]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [8]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)


Epoch 1/5
250/250 [==============================] - 5s 9ms/step - loss: 0.1638 - accuracy: 0.9169 - val_loss: 0.0246 - val_accuracy: 0.9945
Epoch 2/5
250/250 [==============================] - 2s 7ms/step - loss: 0.0152 - accuracy: 0.9970 - val_loss: 0.0248 - val_accuracy: 0.9945
Epoch 3/5
250/250 [==============================] - 2s 7ms/step - loss: 0.0082 - accuracy: 0.9987 - val_loss: 0.0138 - val_accuracy: 0.9975
Epoch 4/5
250/250 [==============================] - 2s 8ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.0315 - val_accuracy: 0.9940
Epoch 5/5
250/250 [==============================] - 2s 8ms/step - loss: 0.0040 - accuracy: 0.9995 - val_loss: 0.0217 - val_accuracy: 0.9965


In [9]:

y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()
print(classification_report(y_test, y_pred))

63/63 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       988
         1.0       1.00      1.00      1.00      1012

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [32]:
def find_similar_songs(txt):
    text_sentiment = custom(txt)
    selected_songs = []
    for _, song in songs.iterrows():
        if abs(song['label'] - text_sentiment) < 0.005:  # adjust threshold as needed
            selected_songs.append((song['song'],song['artist']))
    random.shuffle(selected_songs)
    return selected_songs[:10]

In [11]:
def custom(ct):
    preprocessed_text = preprocess_text(ct)
    custom_seq = tokenizer.texts_to_sequences([preprocessed_text])
    custom_seq = pad_sequences(custom_seq, maxlen=max_length, padding='post', truncating='post')
    # model = keras.models.load_model('model')
    prediction = model.predict(custom_seq)
    return prediction

In [33]:
bruh=input('how are you feeling today')
find_similar_songs(bruh)


1/1 [==============================] - 0s 21ms/step


[('Come Rain or Come Shine', 'Judy Garland'),
 ('Follow Your Heart', 'Triumph'),
 ('You Are the Sunshine of My Life', 'Irene Kral'),
 ('Someday, Somewhere, Somehow', 'Billy Ray Cyrus'),
 ('New Kids on the Block', 'New Kids on the Block'),
 ('Gun the Gold and the Girl', 'Billy Walker'),
 ('Falling Through Your Clothes', 'The New Pornographers'),
 ('My Heart Is Calling', 'Whitney Houston'),
 ('Christmas Spirit', 'Richard Marx'),
 ('The Ballad of Jenny Rae', 'BoDeans')]